<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/5-3%20%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **트리의 앙상블**

## **정형 데이터와 비정형 데이터**

**정형 데이터(structured data)**는 구조화된 데이터입니다. CSV, Database, Excel에 저장하기 쉬운 형태입니다.

**비정형 데이터(unstructured data)**는 비구조화된 데이터입니다. 텍스트 데이터, 사진, 디지털 음악 등을 의미합니다. 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 어렵습니다. 신경망 알고리즘을 활용하여 모델을 만들어야 합니다.

## **앙상블 학습**

**앙상블 학습(Ensemble learning)**의 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 결정 트리 기반 머신러닝 알고리즘입니다. **정형 데이터**를 다루는 데 가장 뛰어난 성과를 냅니다. 참고로 'Ensemble'은 '전체적인 어울림이나 통일', '합주단' 등을 의미합니다.

## **데이터 준비**

모델을 만들 데이터를 준비하겠습니다. 와인 데이터셋입니다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

## **랜덤 포레스트**

**랜덤 포레스트(Random Forest)**는 앙상블 학습을 적용할 때 가장 먼저 시도하길 권장하는 모델이며 매우 안정적인 성능을 냅니다. 결정 트리 여러 개를 랜덤하게 만들어 숲을 만들고, 각 결정 트리의 예측을 사용하여 최종 예측을 만드는 방식을 취합니다. 

각 트리를 훈련하기 위한 데이터는 입력한 훈련 데이터에서 샘플을 랜덤하게 추출하여 만듭니다. 이때 한 샘플이 중복되어 추출될 가능성도 존재합니다. 예컨대 샘플 1,000개가 든 가방에서 샘플 100개를 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣습니다. 계속 100개를 뽑다 보면 중복된 샘플을 뽑을 수도 있습니다. 이렇게 중복되게 뽑은 샘플을 **부트스트랩 샘플(Bootstrap sample)**이라고 부릅니다. **부트스트랩(Bootstrap)**은 보통 **부트스트랩 방식**이라고 부르며, 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식입니다. 기본적으로 부트스트랩 샘플은 훈련 세트 크기와 같게 만듭니다. 즉, 샘플 1,000개가 든 가방에서 샘플 1,000개를 중복하여 뽑습니다. 

아울러 랜덤 포레스트는 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾습니다.

랜덤 포레스트를 구현한 분류 클래스는 `RandomForestClassifier`입니다. 전체 특성 개수의 제곱근만큼 특성을 사용합니다. 다시 말하면 특성이 4개 있다면 노드마다 2개를 랜덤하게 선택하여 사용합니다. 결정 트리의 앙상블이므로 결정 트리를 구현한 클래스인 `DecisionTreeClassifier`가 가진 주요 매개변수인 `criterion`, `max_depth`, `max_feature`, `min_samples_split`, `min_impurity_decrease`, `min_samples_leaf` 등을 모두 제공합니다. 아래는 `RandomForestClassifier` 클래스가 가진 매개변수 목록입니다.

- `n_estimators`: 앙상블을 구성할 트리 개수를 지정하는 매개변수입니다. 기본값은 `100`입니다.

- `criterion`: 불순도를 지정하는 매개변수입니다. 기본값은 지니 불순도를 의미하는 `gini`이며, 엔트로피 불순도를 사용하려면 `entrophy`로 지정합니다.

- `max_depth`: 트리가 성장할 최대 깊이를 지정하는 매개변수입니다. 기본값은 `None`으로 리프 노드가 순수하거나 `min_samples_split` 매개변수에 지정한 값보다 샘플 개수가 적을 때까지 성장합니다.

- `min_samples_split`: 노드를 나누기 위한 최소 샘플 개수를 지정하는 매개변수입니다. 기본값은 `2`입니다.

- `max_features`: 최적의 분할을 위해 탐색할 특성 개수를 지정하는 매개변수입니다. 기본값은 `auto`로 특성 개수의 제곱근을 의미합니다.

- `bootstrap`: 부트스트랩 샘플 사용 여부를 지정하는 매개변수입니다. 기본값은 `True`입니다.

- `oob_score`: OOB 샘플을 사용하여 훈련한 모델을 평가할지 여부를 지정하는 매개변수입니다. 기본값은 `False`입니다. 아래에서 추가로 설명하겠습니다.

- `n_jobs`: 병렬 실행에 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`로 하나의 코어를 사용하며, `-1`로 지정하면 시스템에 있는 모든 코어를 사용합니다.

랜덤 포레스트는 기본적으로 결정 트리 100개를 훈련합니다. 분류일 경우에는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼고, 회귀일 경우에는 각 트리의 예측을 평균한 값을 예측으로 삼습니다. 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 방지하고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있습니다. 종종 기본 매개변수 설정만으로도 좋은 결과를 낼 수 있습니다.

결정 트리의 큰 장점 중 하나인 특성 중요도를 계산하기도 합니다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것입니다.

참고로 `RandomForestRegressor`는 회귀를 다루는 랜덤 포레스트 클래스입니다. `RandomForestClassifier` 클래스가 전체 특성 개수의 제곱근만큼 특성을 사용한다면, `RandomForestRegressor` 클래스는 전체 특성을 사용합니다.

### **교차 검증 수행하기**

교차 검증을 수행하여 최상의 검증 점수를 확인하겠습니다. 매개변수는 `n_jobs`만 `-1`로 바꾸겠습니다.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9973541965122431
검증 세트 점수: 0.8905151032797809


과대적합 상태입니다. 알고리즘을 살펴보는 것이 목적이므로 매개변수를 더 조정하지는 않겠습니다. 이 예제는 매우 간단하고 특성이 많지 않아서 그리드 서치를 사용하더라도 하이퍼파라미터 튜닝 결과는 크게 나아지지 않습니다.

### **모델 훈련 후 특성 중요도 출력하기**

모델을 훈련한 후 특성 중요도를 출력하겠습니다. 특성 중요도는 `feature_importances_` 속성에 저장되어 있습니다.

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


출력된 특성 중요도는 각각 'alcohol', 'sugar', 'pH'를 나타냅니다. 전 챕터에서 결정 트리 모델의 특성 중요도인 `[0.12345626, 0.86862934, 0.0079144]`와 랜덤 포레스트의 특성 중요도를 비교하면 'alcohol'과 'pH'의 중요도가 상승하고 'sugar'의 중요도가 감소했습니다. 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련했기 때문입니다. 특성 하나에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 부여한 것입니다. 이를 통해 과대적합을 줄이고 일반화 성능을 높이는 데 큰 도움이 됩니다.

### **OOB 샘플로 모델 평가하기**

`RandomForestClassifier` 클래스는 자체적으로 모델 평가 점수를 얻을 수 있습니다. 부트스트랩 샘플에 포함되지 않고 남는 샘플을 **OOB(out of bag)**이라고 부르는데, 부트스트랩 샘플로 훈련한 결정 트리를 이 OOB를 사용하여 평가합니다. 검증 세트 역할을 대신한다고 볼 수 있습니다. 교차 검증을 대신하므로 훈련 세트에 더 많은 샘플을 사용할 수 있습니다. `RandomForestClassifier` 클래스의 `oob_score` 매개변수를 `True`로 지정하면 OOB 점수를 평균하여 이 점수를 얻을 수 있습니다.

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수인 `0.8905151032797809`와 비슷한 결과를 얻었습니다.

## **엑스트라 트리**

**엑스트라 트리(Extra Trees)**는 랜덤 포레스트와 매우 비슷하게 작동하는 모델입니다. 기본적으로 결정 트리 100개를 훈련하고, 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용합니다.

아울러 결정 트리를 만들 때 전체 훈련 세트를 사용합니다. 부트스트랩 샘플을 사용하지 않습니다. 대신 노드를 분할할 때 가장 좋은 분할을 찾지 않고 무작위로 분할합니다. 이렇게 노드를 무작위로 분할하면 모델 성능이 낮아지겠지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트 점수를 높이는 효과가 있습니다. 아래에서 추가로 설명하겠습니다.

엑스트라 트리를 실현한 분류 클래스는 `ExtraTreeClassifier`입니다. 결정 트리가 제공하는 매개변수 대부분을 가집니다. 아래는 매개변수 목록입니다.

- `n_estimators`: 앙상블을 구성할 트리 개수를 지정하는 매개변수입니다. 기본값은 `100`입니다.

- `criterion`: 불순도를 지정하는 매개변수입니다. 기본값은 지니 불순도를 의미하는 `gini`이며, 엔트로피 불순도는 `entrophy`입니다.

- `max_depth`: 트리가 성장할 최대 깊이를 지정하는 매개변수입니다. 기본값은 `None`으로 리프 노드가 순수하거나 `min_samples_split` 매개변수에 지정한 값보다 샘플 개수가 적을 때까지 성장합니다.

- `min_samples_split`: 노드를 나누기 위한 최소 샘플 개수를 지정하는 매개변수입니다. 기본값은 `2`입니다.

- `max_features`: 최적의 분할을 위해 탐색할 특성 개수를 지정하는 매개변수입니다. 기본값은 `auto`로 특성 개수의 제곱근을 의미합니다.

- `bootstrap`: 부트스트랩 샘플 사용 여부를 지정하는 매개변수입니다. 기본값은 `False`로 랜덤 포레스트와 차이가 있습니다. 랜덤 포레스트는 `True`입니다.

- `oob_score`: OOB 샘플을 사용하여 훈련한 모델을 평가할지 여부를 지정하는 매개변수입니다. 기본값은 `False`입니다. 

- `n_jobs`: 병렬 실행에 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`로 하나의 코어를 사용하며, `-1`로 지정하면 시스템에 있는 모든 코어를 사용합니다.

기실 `DecisionTreeClassifier` 클래스의 `splitter` 매개변수를 `random`으로 지정하면 엑스트라 트리와 동일합니다. 이전 챕터에서 수행했던 방식입니다.

참고로 `ExtraTreesRegressor` 클래스는 엑스트라 트리를 실현한 회귀 클래스입니다.

### **교차 검증 수행하기**

교차 검증을 수행하겠습니다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9974503966084433
검증 세트 점수: 0.8887848893166506


랜덤 포레스트와 비슷한 점수를 얻었습니다. 이 예제는 특성이 많지 않아서 두 모델의 차이가 크지 않습니다.

보통 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련하지만 노드를 랜덤하게 분할하므로 계산 속도가 빠릅니다.

### **모델 훈련 후 특성 중요도 출력하기**

특성 중요도를 출력하겠습니다.

In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


출력된 특성 중요도는 각각 'alcohol', 'sugar', 'pH'를 나타냅니다. 전 챕터에서 결정 트리 모델의 특성 중요도인 `[0.12345626, 0.86862934, 0.0079144]`와 엑스트라 트리의 특성 중요도와 비교하면 'alcohol'과 'pH'의 중요도가 상승하고 'sugar'의 중요도는 감소했습니다.

## **그레이디언트 부스팅**

**그레이디언트 부스팅(Gradient boosting)**은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법입니다. '그레이디언트(Gradient)'의 '경사', '기울기'라는 의미처럼 **경사 하강법**을 사용하여 트리를 앙상블에 추가합니다. 이전에 설명했듯이 경사 하강법은 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정입니다. 

그레이디언트 부스팅은 모델의 가중치와 절편을 조금씩 바꾸면서 가장 낮은 곳으로 내려옵니다. 손실 함수의 낮은 곳으로 천천히 조금씩 이동하기 위해 깊이가 얕은 결정 트리를 계속 추가하면서 이동하는 방법을 취합니다. 분류에서는 로지스틱 손실 함수를 사용하고, 회귀에서는 평균 제곱 오차 함수를 사용합니다.

그레이디언트 부스팅을 실현한 분류 클래스는 `GradientBoostingClassifier`입니다. 기본적으로 깊이가 3인 결정 트리를 100개 사용합니다. 깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 높은 일반화 성능을 낼 수 있습니다. 아래는 `GradientBoostingClassifier` 클래스가 가진 매개변수 목록입니다.

- `loss`: 손실 함수를 지정하는 매개변수입니다. 기본값은 `deviance`로 로지스틱 손실 함수를 의미합니다.

- `n_estimators`: 부스팅 단계를 수행하는 트리 개수를 지정하는 매개변수입니다. 기본값은 `100`입니다.

- `learning_rate`: 트리가 앙상블에 기여하는 정도인 학습률을 지정하는 매개변수입니다. 속도를 조절할 수 있습니다. 기본값은 `0.1`입니다.

- `subsample`: 트리 훈련에 사용하는 훈련 세트의 비율을 정하는 매개변수입니다. 기본값은 `1.0`으로 전체 훈련 세트를 사용합니다. `1.0`보다 작으면 훈련 세트 일부를 사용하며, 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷합니다.

- `max_depth`: 개별 회귀 트리의 최대 깊이를 지정하는 매개변수입니다. 기본값은 `3`입니다.

일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 내지만, 순서대로 트리를 추가하기 때문에 훈련 속도가 느립니다. `GradientBoostingClassifier` 클래스에는 `n_jobs` 매개변수가 없습니다.

참고로 그레이디언트 부스팅을 구현한 회귀 클래스는 `GradientBoostingRegressor`입니다.

### **교차 검증 수행하기**

교차 검증을 수행하겠습니다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.8881086892152563
검증 세트 점수: 0.8720430147331015


과대적합이 거의 되지 않았습니다. 그레이디언트 부스팅은 결정 트리 개수를 늘려도 과대적합에 매우 강합니다. 학습률을 증가시키고 트리 개수를 늘리면 성능이 더 향상될 수 있습니다.

### **매개변수를 조정하여 점수 출력하기**

`n_estimators` 매개변수 값을 변경하여 결정 트리 개수를 `500`개로 늘리고, `learning_rate` 매개변수 값을 변경하여 학습률을 `0.2`로 지정하여 점수를 출력해보겠습니다.

In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True,
                        n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9464595437171814
검증 세트 점수: 0.8780082549788999


과대적합을 최대한 억제하고 있습니다. 하지만 두 점수간 차이는 꽤 큰 편으로 볼 수 있습니다.

### **모델 훈련 후 특성 중요도 출력하기**

In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


출력된 특성 중요도는 각각 'alcohol', 'sugar', 'pH'를 나타냅니다. 랜덤 포레스트 모델의 특성 중요도인 `[0.23167441 0.50039841 0.26792718]`와 그레이디언트 부스팅의 특성 중요도를 비교하면, 그레이디언트 부스팅은 랜덤 포레스트보다 'sugar'에 더 집중하고, 나머지 두 특성에는 덜 집중합니다.

## **히스토그램 기반 그레이디언트 부스팅**

**히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)**은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘입니다. 입력 특성을 256개 구간으로 나눠서 노드를 분할할 때 최적 분할을 매우 빠르게 찾습니다. 256개 구간 중에 하나를 떼어 놓고 누락된 값을 위해 사용하며, 입력에 누락된 특성이 있더라도 따로 전처리할 필요가 없습니다.

히스토그램 기반 그레이디언트 부스팅을 구현한 분류 클래스는 `HistGradientBoostingClassifier`입니다. 일반적으로 기본 매개변수만으로도 안정적인 성능을 얻을 수 있습니다. 아래는 관련 매개변수 목록입니다.

- `learning_rate`: 학습률(감쇠율)을 지정하는 매개변수입니다. '감쇠'는 '힘이 줄어서 약해짐'을 의미함. 기본값은 `0.1`이며, `1.0`이면 감쇠가 전혀 없습니다.

- `max_iter`: 부스팅 단계를 수행하는 트리 개수(부스팅 반복 횟수)를 지정하는 매개변수입니다. 기본값은 `100`입니다. 모델 성능을 높이려면 이 매개변수를 테스트합니다. 트리 개수를 지정하는 `n_estimators` 매개변수 대신 사용합니다.

- `max_bins`: 입력 데이터를 나눌 구간 개수를 지정하는 매개변수입니다. 기본값은 `255`이며 최댓값이므로 더 크게 지정할 수는 없습니다. 위에서 설명한 구간 256개는 누락된 값을 위해 구간 1개가 추가된 것입니다.

`HistGradientBoostingRegressor` 클래스는 히스토그램 기반 그레이디언트 부스팅을 구현한 회귀 클래스입니다.

### **교차 검증 수행하기**

히스토그램 기반 그레이디언트 부스팅은 아직 테스트 과정에 속해 있으며, 사용하려면 `sklearn.experimental` 패키지에 있는 `enable_hist_gradient_boosting` 모듈을 임포트 해야 합니다(사이킷런 1.0에서 히스토그램 기반 부스팅이 `experimental` 모듈에서 벗어났으므로 이 모듈을 임포트 할 필요는 없습니다). 교차 검증을 수행해보겠습니다.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9321723946453317
검증 세트 점수: 0.8801241948619236


과대적합을 억제하면서 그레이디언트 부스팅보다 약간 높은 성능을 보여줍니다.

### **모델 훈련 후 특성 중요도 출력하기**

특성 중요도를 계산하려면 `permutation_importance()` 메서드를 사용합니다. 'permutation'은 '순열, 치환'을 의미합니다. 특성을 하나씩 랜덤하게 섞고 모델 성능이 변화하는지 관찰하면서 어떤 특성이 중요한지를 계산합니다. 훈련 세트뿐만 아니라 테스트 세트에서도 적용할 수 있고, 사이킷런에서 제공하는 추정기 모델에도 적용 가능할 수 있습니다. 아래는 `permutation_importance()` 메서드가 가진 매개변수입니다.

- `n_repeats`: 랜덤하게 섞을 횟수를 지정하는 매개변수입니다. 기본값은 `5`입니다.

이외 매개변수는 `help(permutation_importance)`로 확인하시기 바랍니다.

속성은 3가지를 가집니다.

- `improtances`: 반복하여 얻은 특성 중요도입니다.

- `importances_mean`: 반복하여 얻은 특성 중요도의 평균입니다.

- `importances_std`: 반복하여 얻은 특성 중요도의 표준편차입니다.

모델을 훈련하고 특성 중요도를 계산하겠습니다. 

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(f'특성 중요도:\n{result.importances}')
print()
print(f'특성 중요도 평균:     {result.importances_mean}')
print(f'특성 중요도 표준편차: {result.importances_std}')

특성 중요도:
[[0.08793535 0.08350972 0.08908986 0.08312488 0.09274581 0.08755051
  0.08601116 0.09601693 0.09082163 0.09082163]
 [0.22782374 0.23590533 0.23936887 0.23436598 0.23725226 0.23436598
  0.23359631 0.23398114 0.23994612 0.22724649]
 [0.08581874 0.08601116 0.08062344 0.07504329 0.08427939 0.07792957
  0.07234943 0.07465846 0.08139311 0.08466423]]

특성 중요도 평균:     [0.08876275 0.23438522 0.08027708]
특성 중요도 표준편차: [0.00382333 0.00401363 0.00477012]


평균은 랜덤 포레스트와 비슷한 비율을 나타냅니다. 

테스트 세트로 특성 중요도를 계산하겠습니다.

In [ ]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(f'특성 중요도:\n{result.importances}')
print()
print(f'특성 중요도 평균:     {result.importances_mean}')
print(f'특성 중요도 표준편차: {result.importances_std}')

특성 중요도:
[[0.06230769 0.05769231 0.05538462 0.05538462 0.06076923 0.06076923
  0.06846154 0.06230769 0.05461538 0.05923077]
 [0.20076923 0.2        0.21153846 0.20076923 0.20307692 0.18923077
  0.19615385 0.19461538 0.21384615 0.21384615]
 [0.05692308 0.04692308 0.05076923 0.04769231 0.04692308 0.05
  0.04384615 0.04692308 0.04307692 0.05692308]]

특성 중요도 평균:     [0.05969231 0.20238462 0.049     ]
특성 중요도 표준편차: [0.004      0.007938   0.00453846]


그레이디언트 부스팅과 비슷하게 좀 더 'sugar'에 집중하고 있습니다. 이러한 과정을 통해 어떤 특성에 중점을 둘지 예상할 수 있습니다.

### **테스트 세트로 성능 확인하기**

테스트 세트로 정확도를 확인하겠습니다.

In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

실전에 투입하면 이보다 더 낮은 성능을 가질 가능성이 높지만, 결론적으로 앙상블 모델은 단일 결정 트리 모델보다 좋은 결과를 얻었습니다. 2절 랜덤 서치에서 테스트 세트의 정확도는 86%였습니다.

### **XGBoost**

**XGBoost**는 트리 기반 앙상블 학습에서 가장 각광받는 알고리즘 중 하나입니다. `cross_validate()` 메서드와 함께 사용할 수도 있고, `tree_method` 매개변수를 `hist`로 지정하여 히스토그램 기반 그레이디언트 부스팅을 사용할 수도 있습니다. 실전에서 매우 활발하게 사용합니다. 더 자세한 내용은 [링크1](https://xgboost.ai/), [링크2](https://xgboost.readthedocs.io/en/latest/)를 참고하시기 바랍니다.

#### **교차 검증 수행하기**

교차 검증을 수행하겠습니다. `tree_method` 매개변수를 `hist`로 지정하여 히스토그램 기반 그레이디언트 부스팅을 사용하겠습니다.

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.8824322471423747
검증 세트 점수: 0.8726214185237284


#### **모델 훈련 후 특성 중요도 출력하기**

모델을 훈련하고 특성 중요도를 출력하겠습니다.

In [ ]:
from sklearn.inspection import permutation_importance

xgb.fit(train_input, train_target)
result = permutation_importance(xgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)

print(f'특성 중요도:\n{result.importances}')
print()
print(f'특성 중요도 평균:     {result.importances_mean}')
print(f'특성 중요도 표준편차: {result.importances_std}')

특성 중요도:
[[0.03983067 0.03521262 0.04329421 0.039061   0.0421397  0.03925342
  0.03713681 0.04348663 0.04252453 0.04310179]
 [0.19607466 0.20069271 0.19973061 0.20030787 0.20319415 0.20319415
  0.19665191 0.19395805 0.20261689 0.19415047]
 [0.03675197 0.03636713 0.0359823  0.03232634 0.03790648 0.03617472
  0.03386569 0.02944006 0.03867616 0.03790648]]

특성 중요도 평균:     [0.04050414 0.19905715 0.03553973]
특성 중요도 표준편차: [0.00271338 0.00341103 0.00272264]


#### **테스트 세트로 성능 확인하기**

테스트 세트로 정확도를 확인하겠습니다.

In [ ]:
xgb.score(test_input, test_target)

0.8669230769230769

### **LightGBM**

**LightGBM**은 XGBoost와 함께 부스팅 계열 알고리즘에서 가장 각광을 받습니다. XGBoost처럼 `cross_validate()` 메서드와 함께 사용할 수도 있고, `tree_method` 매개변수를 `hist`로 지정하여 히스토그램 기반 그레이디언트 부스팅을 사용할 수도 있습니다. 실전에서 매우 활발하게 사용합니다.

XGBoost와 비교하면 성능상에서는 큰 차이가 없지만, 2년 늦게 만들어졌기에 몇 가지 장점을 가집니다. 더 빠른 학습 속도와 예측 수행 시간, 더 작은 메모리 사용량, 카테고리형 특성을 자동으로 변환하고 최적으로 분할하는 것입니다. 더 자세한 설명은 [링크1](https://github.com/microsoft/LightGBM), [링크2](https://lightgbm.readthedocs.io/en/latest)를 참고하시기 바랍니다.

#### **교차 검증 수행하기**

교차 검증을 수행하겠습니다. 모든 방식은 `XGBClassifier` 클래스를 사용하는 과정과 동일합니다.

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(tree_method='hist', random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True)

print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9338079582727165
검증 세트 점수: 0.8789710890649293


#### **모델 훈련 후 특성 중요도 출력하기**

모델을 훈련하고 특성 중요도를 출력하겠습니다.

In [ ]:
from sklearn.inspection import permutation_importance

lgb.fit(train_input, train_target)
result = permutation_importance(lgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)

print(f'특성 중요도:\n{result.importances}')
print()
print(f'특성 중요도 평균:     {result.importances_mean}')
print(f'특성 중요도 표준편차: {result.importances_std}')

특성 중요도:
[[0.08908986 0.08389455 0.09024437 0.08524149 0.09582451 0.09101405
  0.09005195 0.09390033 0.09274581 0.09332307]
 [0.23051761 0.23513566 0.24110063 0.23224937 0.2376371  0.23609775
  0.23571291 0.2345584  0.24013854 0.22705407]
 [0.08716567 0.08697325 0.08774293 0.0802386  0.0894747  0.07869925
  0.07562055 0.07869925 0.08735809 0.08601116]]

특성 중요도 평균:     [0.090533   0.2350202  0.08379835]
특성 중요도 표준편차: [0.00355975 0.00404098 0.00467245]


#### **테스트 세트로 성능 확인하기**

테스트 세트로 정확도를 확인하겠습니다.

In [ ]:
lgb.score(test_input, test_target)

0.8638461538461538